In [25]:
import torch
print("🧠 PyTorch - GPU disponible:", torch.cuda.is_available())

import tensorflow as tf
print("⚙️ TensorFlow - GPU detectada:", tf.config.list_physical_devices('GPU'))


🧠 PyTorch - GPU disponible: True
⚙️ TensorFlow - GPU detectada: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Solo errores, no warnings ni info

# === Módulos estándar ===
import numpy as np
import random
import time

# === Keras ===
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, Input # type: ignore
from tensorflow.keras.initializers import GlorotUniform # type: ignore
from tensorflow.keras.optimizers import SGD as KSGD, Adam as KAdam # type: ignore
from tensorflow.keras.losses import SparseCategoricalCrossentropy # type: ignore

# === PyTorch ===
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# === Sklearn ===
from sklearn.neural_network import MLPClassifier

# === other imports ===
import numpy as np, csv, os, time, itertools
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import glob, os, math, io, imageio, pandas as pd
from tqdm import tqdm

E0000 00:00:1750382250.186038  112172 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750382250.217477  112172 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750382250.453314  112172 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750382250.453392  112172 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750382250.453394  112172 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750382250.453395  112172 computation_placer.cc:177] computation placer already registered. Please check linka

In [27]:
def make_model(framework, seed):
    random.seed(seed)
    np.random.seed(seed)

    if framework == "keras":
        tf.random.set_seed(seed)
        model = Sequential([
            Dense(1024, activation='relu', kernel_initializer=GlorotUniform(seed), input_shape=(32*32*3,)),
            Dense(512,  activation='relu', kernel_initializer=GlorotUniform(seed)),
            Dense(256,  activation='relu', kernel_initializer=GlorotUniform(seed)),
            Dense(128,  activation='relu', kernel_initializer=GlorotUniform(seed)),
            Dense(10,   activation='softmax',  # ← salida softmax, 5.ª capa
                kernel_initializer=GlorotUniform(seed))
        ])
        return model


    elif framework == "torch":
        torch.manual_seed(seed)
        model = nn.Sequential(
            nn.Linear(3072, 1024), nn.ReLU(),   
            nn.Linear(1024, 512),  nn.ReLU(),   
            nn.Linear(512, 256),   nn.ReLU(),   
            nn.Linear(256, 128),   nn.ReLU(),   
            nn.Linear(128, 10)                 
        )
        return model


    elif framework == "sklearn":
        model = MLPClassifier(
            hidden_layer_sizes=(1024, 512, 256, 128),
            activation='relu',
            solver='adam',
            max_iter=1,
            random_state=seed
        )
        return model

    else:
        raise ValueError(f"Framework '{framework}' no soportado.")


In [28]:
import os, time, numpy as np
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import SGD as KSGD, Adam as KAdam
import torch.nn as nn, torch.optim as optim, torch
from sklearn.neural_network import MLPClassifier
import joblib

def train_network(network, X, y, optimizer, learning_rate, batch_size, timeout):
    results = []
    start_time = time.time()

    # Crea carpeta si no existe
    os.makedirs("models", exist_ok=True)

    # Etiqueta común para el archivo
    tag = f"{optimizer}_bs{batch_size}_lr{learning_rate}"

    print(f"🔍 Detectando tipo de red: {type(network)}")

    # ================ KERAS ================
    try:
        if hasattr(network, "train_on_batch") and hasattr(network, "compile"):
            print("✅ Red Keras detectada. Iniciando entrenamiento…")
            network.compile(
                optimizer=KAdam(learning_rate) if optimizer == "adam" else KSGD(learning_rate),
                loss=SparseCategoricalCrossentropy(),
            )

            num_samples = X.shape[0]
            while True:
                if time.time() - start_time >= timeout - 0.1:
                    break
                idx = np.random.choice(num_samples, batch_size, replace=False)
                loss = network.train_on_batch(X[idx], y[idx])
                elapsed = round(time.time() - start_time, 4)
                results.append((elapsed, float(loss)))

            # ─── Guardar modelo ───
            path = f"models/keras_{tag}.keras"
            network.save(path)
            print(f"💾 Modelo Keras guardado en {path}")
            return results
    except Exception as e:
        print("❌ Error en bloque Keras:", e)

    # ================ PYTORCH ================
    try:
        if isinstance(network, nn.Module):
            print("✅ Red PyTorch detectada. Iniciando entrenamiento…")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            network.to(device).train()

            X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
            y_tensor = torch.tensor(y, dtype=torch.long).to(device)
            opt      = optim.Adam(network.parameters(), lr=learning_rate) if optimizer == "adam" else optim.SGD(network.parameters(), lr=learning_rate)
            loss_fn  = nn.CrossEntropyLoss()

            num_samples = X.shape[0]
            while True:
                if time.time() - start_time >= timeout - 0.1:
                    break
                idx = np.random.choice(num_samples, batch_size, replace=False)
                opt.zero_grad()
                loss = loss_fn(network(X_tensor[idx]), y_tensor[idx])
                loss.backward(); opt.step()
                elapsed = round(time.time() - start_time, 4)
                results.append((elapsed, float(loss.item())))

            # ─── Guardar modelo ───
            path = f"models/torch_{tag}.pt"
            torch.save(network.state_dict(), path)
            print(f"💾 Modelo PyTorch guardado en {path}")
            return results
    except Exception as e:
        print("❌ Error en bloque PyTorch:", e)

    # ================ SKLEARN ================
    try:
        if isinstance(network, MLPClassifier):
            print("✅ Red sklearn detectada. Iniciando entrenamiento…")
            num_samples = X.shape[0]
            classes     = np.unique(y)
            while True:
                if time.time() - start_time >= timeout - 0.1:
                    break
                idx = np.random.choice(num_samples, batch_size, replace=False)
                network.partial_fit(X[idx], y[idx], classes=classes)

                # estimar pérdida
                probs = network.predict_proba(X[idx])
                loss  = -np.mean(np.log(probs + 1e-9)[np.arange(len(idx)), y[idx]])
                elapsed = round(time.time() - start_time, 4)
                results.append((elapsed, float(loss)))

            # ─── Guardar modelo ───
            path = f"models/sklearn_{tag}.pkl"
            joblib.dump(network, path)
            print(f"💾 Modelo sklearn guardado en {path}")
            return results
    except Exception as e:
        print("❌ Error en bloque sklearn:", e)

    print("⚠️ Ningún framework compatible fue detectado.")
    raise ValueError(f"Tipo de red no reconocido: {type(network)}")


In [29]:
# --- carga CIFAR-10 y aplana ---
(X_tr, y_tr), _ = cifar10.load_data()
X_tr = X_tr.reshape((X_tr.shape[0], -1)).astype("float32") / 255.0
y_tr = y_tr.flatten()


# frameworks  = ["keras", "torch", "sklearn"]
# batch_sizes = [16, 64, 256, 2048]
# optimizers  = ["sgd", "adam"]
# lrs         = [0.001, 0.01]


frameworks  = ["keras", "torch", "sklearn"]
batch_sizes = [256]
optimizers  = ["adam"]
lrs         = [0.001]

out_dir = "csv_logs";  os.makedirs(out_dir, exist_ok=True)

for fw, bs, opt, lr in itertools.product(frameworks, batch_sizes, optimizers, lrs):
    tag = f"{fw}_bs{bs}_{opt}_lr{lr}"
    print("▶️  Ejecutando", tag, flush=True)

    model = make_model(fw, seed=7)
    results = train_network(model, X_tr, y_tr,
                            optimizer=opt,
                            learning_rate=lr,
                            batch_size=bs,
                            timeout=10)       # 10 min

    with open(os.path.join(out_dir, tag + ".csv"), "w", newline="") as f:
        wr = csv.writer(f)
        wr.writerow(["time_s", "loss"])
        wr.writerows(results)


▶️  Ejecutando keras_bs256_adam_lr0.001
🔍 Detectando tipo de red: <class 'keras.src.models.sequential.Sequential'>
✅ Red Keras detectada. Iniciando entrenamiento…


/home/pipe/anaconda3/envs/deeplearning/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


💾 Modelo Keras guardado en models/keras_adam_bs256_lr0.001.keras
▶️  Ejecutando torch_bs256_adam_lr0.001
🔍 Detectando tipo de red: <class 'torch.nn.modules.container.Sequential'>
✅ Red PyTorch detectada. Iniciando entrenamiento…
💾 Modelo PyTorch guardado en models/torch_adam_bs256_lr0.001.pt
▶️  Ejecutando sklearn_bs256_adam_lr0.001
🔍 Detectando tipo de red: <class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>
✅ Red sklearn detectada. Iniciando entrenamiento…
💾 Modelo sklearn guardado en models/sklearn_adam_bs256_lr0.001.pkl


In [30]:
def smooth(y, k=30):
    return pd.Series(y).rolling(window=k, min_periods=1).mean().values

def gif_config_progress(config_tag, n_frames=200, fps=12, y_max=5):
    """
    Genera un GIF que compara Keras, PyTorch y sklearn para una misma configuración.
    `config_tag` es la parte común del nombre, ej.: 'bs16_sgd_lr0.001'
    """
    FRAMEWORKS = ["keras", "torch", "sklearn"]
    curves, max_time = [], 0
    for fw in FRAMEWORKS:
        path = f"csv_logs/{fw}_{config_tag}.csv"
        if not os.path.exists(path):
            print(f"⚠️  Falta {path} (se omitirá)")
            continue
        df = pd.read_csv(path)
        t, l = df["time_s"].values, df["loss"].values
        curves.append((fw.capitalize(), t, l))
        max_time = max(max_time, t[-1])

    if len(curves) < 2:
        print("❌ Se necesitan al menos 2 frameworks para comparar.")
        return

    frame_times = np.linspace(0, max_time, n_frames)
    frames = []
    print(f"🎞️  Generando {n_frames} frames para {config_tag}…")
    for t_cut in tqdm(frame_times):
        fig, ax = plt.subplots(figsize=(8,5))
        for label, t, l in curves:
            idx = np.searchsorted(t, t_cut, side="right")
            if idx > 1:
                # suavizamos solo lo que vamos a dibujar
                l_smooth = smooth(l[:idx])
                t_smooth = t[:len(l_smooth)]
                ax.plot(t_smooth, l_smooth, label=label, linewidth=2)

        ax.set(xlim=(0, math.ceil(max_time)),
               ylim=(0, y_max),
               xlabel="Tiempo [s]",
               ylabel="Pérdida",
               title=f"Comparación de frameworks – {config_tag} – t={t_cut:0.1f}s")
        ax.grid(alpha=0.3)
        ax.legend(fontsize=10)
        fig.tight_layout()

        buf = io.BytesIO()
        fig.savefig(buf, format="png")
        plt.close(fig)
        buf.seek(0)
        frames.append(imageio.v2.imread(buf))

    gif_name = f"compare_{config_tag}.gif"
    imageio.mimsave(gif_name, frames, fps=fps)
    print("✅ GIF guardado en", gif_name)

In [31]:
gif_config_progress("bs256_adam_lr0.001")

🎞️  Generando 200 frames para bs256_adam_lr0.001…


  0%|          | 0/200 [00:00<?, ?it/s]

/tmp/ipykernel_1164/2073557707.py:44: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(fontsize=10)
100%|██████████| 200/200 [00:23<00:00,  8.34it/s]


✅ GIF guardado en compare_bs256_adam_lr0.001.gif


In [32]:
# def gif_framework_progress(framework, n_frames=200, fps=12):
#     csv_files = sorted(glob.glob(f"csv_logs/{framework}_*.csv"))
#     if not csv_files:
#         print("❌ No se encontraron CSV para", framework)
#         return
    
#     # Cargar todas las curvas
#     curves, max_time = [], 0
#     for path in csv_files:
#         df = pd.read_csv(path)
#         t, l = df["time_s"].values, df["loss"].values
#         curves.append((os.path.basename(path).replace(".csv", ""), t, l))
#         max_time = max(max_time, t[-1])
    
#     frame_times = np.linspace(0, max_time, n_frames)
#     frames = []
#     print(f"🎞️  Generando {n_frames} frames para {framework}…")
#     for t_cut in tqdm(frame_times):
#         fig, ax = plt.subplots(figsize=(12,8))
#         for label, t, l in curves:
#             idx = np.searchsorted(t, t_cut, side="right")
#             if idx > 1:
#                 ax.plot(t[:idx], l[:idx], label=label, linewidth=1)
#         ax.set(xlim=(0, math.ceil(max_time)), ylim=(0,10),
#                xlabel="Tiempo [s]", ylabel="Pérdida",
#                title=f"{framework.upper()} – hasta {t_cut:0.1f}s")
#         ax.grid(alpha=0.3)
#         ax.legend(fontsize=6, ncol=2, loc="upper right", frameon=False)
#         fig.tight_layout()
        
#         # ─── convertir figura a array ───
#         buf = io.BytesIO()
#         fig.savefig(buf, format="png")       # guarda figura en memoria
#         plt.close(fig)
#         buf.seek(0)
#         frames.append(imageio.v2.imread(buf))  # lee PNG desde el buffer

#     gif_name = f"{framework}_all_configs.gif"
#     imageio.mimsave(gif_name, frames, fps=fps)
#     print("✅ GIF guardado en", gif_name)

# # Ejemplo de uso
# gif_framework_progress("keras")   # o "torch", "sklearn"


In [2]:
import glob, os, pandas as pd
import numpy as np

CSV_DIR = "csv_logs"   # carpeta donde guardas los *.csv
K_LAST  = 10           # cuántos pasos finales promediar

def avg_last_losses(path, k=K_LAST):
    """Promedio de las k últimas pérdidas del CSV (o menos si hay pocos datos)."""
    df = pd.read_csv(path)
    return float(df["loss"].tail(k).mean()) if not df.empty else np.inf

def top_configs(csv_dir=CSV_DIR, k_last=K_LAST, top_n=10):
    entries = []
    for csv_path in glob.glob(os.path.join(csv_dir, "*.csv")):
        tag  = os.path.basename(csv_path).replace(".csv", "")  # nombre sin extensión
        loss = avg_last_losses(csv_path, k=k_last)
        if np.isfinite(loss):
            entries.append((loss, tag))        # (valor, configuración)

    # Ordena por menor pérdida y toma los top_n
    return sorted(entries, key=lambda x: x[0])[:top_n]

if __name__ == "__main__":
    top10 = top_configs(top_n=10)
    if top10:
        print(f"\n🏆  TOP 10 configuraciones (promedio de las últimas {K_LAST} pérdidas):")
        for rank, (loss, tag) in enumerate(top10, start=1):
            print(f"{rank:2d}. {tag:<40}  avg_loss = {loss:.4f}")
    else:
        print("No se encontraron archivos CSV en", CSV_DIR)


🏆  TOP 10 configuraciones (promedio de las últimas 10 pérdidas):
 1. keras_bs256_sgd_lr0.01                    avg_loss = 0.4088
 2. keras_bs64_sgd_lr0.01                     avg_loss = 0.5339
 3. keras_bs64_adam_lr0.001                   avg_loss = 0.6213
 4. keras_bs2048_adam_lr0.001                 avg_loss = 0.6849
 5. sklearn_bs2048_adam_lr0.001               avg_loss = 0.7162
 6. sklearn_bs2048_sgd_lr0.01                 avg_loss = 0.7169
 7. sklearn_bs2048_sgd_lr0.001                avg_loss = 0.7178
 8. sklearn_bs2048_adam_lr0.01                avg_loss = 0.7359
 9. keras_bs16_sgd_lr0.01                     avg_loss = 0.9728
10. sklearn_bs256_sgd_lr0.01                  avg_loss = 1.0331
